# Parquet Analytics Demo Notebook

This notebook serves as a brief demonstration of how a set of cloud-hosted parquet files can be analyzed locally using R. While the analysis in this notebook consists of contrived / toy examples, the data being operated on is in fact _real_! It is currently stored in the FHIR server as part of the PHDI Ingestion Pipeline and has been transformed and standardized by a number of preceding building blocks.

To run this notebook, you'll need to configure your system to recognize the R kernel. A simple tutorial can be found here: https://irkernel.github.io/installation/. To start, make sure you've installed the latest version of R, available from the CRAN repository at https://cloud.r-project.org/index.html. Then, launch R from the terminal with the `R` command. **Important:** We strongly recommend installing R from CRAN and performing the next step with R launched from your terminal. This is because the R executable libraries install in different places on your computer  depending on whether you installed R from CRAN, the R app / RStudio, or a package manager like homebrew, which can make make modifying the `PATH` environment variable complicated. Performing the next step using a terminal-launched R will install all of the relevant frameworks and executables in the same place without authorization issues. Run `install.packages('IRkernel')`, then run `IRkernel::installspec()`. That's it! Your notebook software can now "see" the R kernel on your machine.

## Packages and imports

We'll begin by installing some analytics packages that we'll use for our examples. We'll also need two packages that allow us to interact with and access Microsoft Azure's storage platform. We'll need:

* `arrow`
* `ggplot2`
* `jsonlite`
* `geojsonio`
* `sf`
* `tidyverse`
* `AzureStor`
* `AzureRMR`

which you can install by running `install.packages(c("AzureStor", "AzureRMR", "arrow", "ggplot2", "jsonlite", "geojsonio", "sf", "tidyverse")`. These packages allow us to interact with Microsoft Azure, load parquet data, interact with and format data the way we want, and visualize our result. If you're using a different cloud platform for file storage and management, you may need other packages instead of these, but for our demo, we'll use these two to handle connecting with Azure.

While common practice is to use the `rgdal` package for spatial analytics purposes (see last example), `rgdal` will reach its end-of-supported lifecycle in early 2023. Hence, we recommend the new industry standard `sf` package (standing for "simple features").

In [ ]:
# Import relevant packages
library(arrow)
library(jsonlite)
library(AzureStor)
library(AzureRMR)
library(sf)
library(geojsonio)
library(tidyverse)

## Configuring variables and our notebook environment

Before we get into the code proper, we need to configure a few variables so that things run smoothly. There are four variables that correspond to accessing a cloud-hosted platform directly through R. They will, respectively, allow us to make an authorized connection, to a particular storage provider, so that we can access the endpoint where the files live, within a specific storage container. There's also one variable that holds a file path for tract files for the state of Virginia (more on that in our last example). For this particular implementation, we're hosting files in an Azure Gen. 2 storage bucket, but the principles are the same regardless of the hosting platform.

In [ ]:
TENANT_ID = "YOUR TENANT ID STRING"
HOST = "URL TO YOUR STORAGE PROVIDER" # for example, https://storage.azure.com/ -- MUST end with forward slash!
ADLS_ENDPOINT = "YOUR SPECIFIC ENDPOINT"
CONTAINER = "NAME OF YOUR STORAGE CONTAINER"
PATH_TO_TRACT_FILES = "ABSOLUTE FILE PATH TO VA TRACT DIVISIONS SHAPEFILE"

For ease of visualization, we'll also modify the default graphics window that R uses within jupyter notebook, so that our graphs are easier to see. Note that these values can be modified to anything you'd like; the values in right now are just ones that we feel help demo the visualizations.

In [ ]:
options(repr.plot.width=10, repr.plot.height=8)

## Access and download data

Now that we've got our package environment set up, let's load the data. All of the data used in this notebook is real ECR data from Virginia; we've used our intake pipeline to standardize and transform the data into FHIR resources before eventually converting it to a parquet output (each of these tasks was accomplished using one or more of our building blocks). This notebook demonstrates the important fact that the output of the PHDI Building Blocks pipeline are cloud-hosted files that are accessible locally for all manner of analytics. Let's access the Storage Account and pull down the parquet data to read into R.

In [ ]:
# Configure container access to Azure storage
az <- create_azure_login(tenant=TENANT_ID, host=HOST)
token <- az$token$credentials$access_token
adls_ep <- adls_endpoint(ADLS_ENDPOINT, token=token)
container <- storage_container(adls_ep, CONTAINER)

# Initialize some tempfiles to buffer data into
phdi_conditions = tempfile()
phdi_organizations = tempfile()
phdi_compositions = tempfile()
phdi_patients = tempfile()

# Perform storage download, copying data into the temp files
storage_download(container, "condition.parquet", phdi_conditions)
storage_download(container, "organization.parquet", phdi_organizations)
storage_download(container, "composition.parquet", phdi_compositions)
storage_download(container, "patient.parquet", phdi_patients)

# Notice that we can read-in only a subset of columns if we so choose
# i.e. It's not necessary to fully read the file to memory
conditions = read_parquet(phdi_conditions, col_select = c("patient_id", "onset_date"))
organizations = read_parquet(phdi_organizations)
compositions = read_parquet(phdi_compositions)
patients = read_parquet(phdi_patients)

We've now created local copies of all data that maps into the ECR schema--we have resources for Patients, Conditions, Organizations, and Compositions (between patients and organizations). Importantly, we've stored all of this data into `tempfiles` which will persist _only_ as long as our current R session. That way, when we're finished with analysis, the files will be cleared, leaving no chance for PII to linger. We'll use these file for the remainder of our analysis demo, noting that pulling files down this way allows analysts to experiment freely on data without any risk of corrupting the source of truth.

Before getting into some demo analytics problems, we need to do a bit of data modification and format. If we try to visualize the `patient` data frame, we'll get an error describing that the expected length of a display string should be one value, but a different value was encountered by the print formatting function (the exact values themselves aren't important, it's the discrepancy in the numbers that breaks the display):

In [ ]:
# This call WILL result in an error
head(patients)

This is because the `phone_number` column is not only a nested list, but because the information stored within that column is of variable length (because some patients have phone numbers on file while others do not). Let's format this column to line up with what we'd like our data frame to be: a singular, flattened entity on which we can do joins and perform analysis.

In [ ]:
# Flatten the list column to a series of regular columns
patients <- patients %>% 
    unnest(phone_number) %>%
    subset(select = -c(system, use)) %>%
    rename(phone_number = value)

# Only display the columns of the data frame that don't contain PII
head(patients[,c("patient_id", "gender", "state", "country")])

### Joining Data

The first thing we'll likely want to do with our data is join it. We have separate tables for each of the various resources we pulled down in our schema, but if we want to answer questions _across_ the data, we'll need to connect the dots. The `patient_id` field uniquely generated for all patients in the database is a great way of doing just this. Let's run joins to attach our other resources to the patients they're associated with.

In [ ]:
# Perform join--patients and conditions share a patient_id field,
# and we can use the composition to associate patients with the
# organizations they're associated to
joined_df <- patients %>%
    left_join(conditions, by = "patient_id") %>%
    left_join(compositions, by = "patient_id") %>%
    left_join(organizations, by = "organization_id")
colnames(joined_df)
head(joined_df[,c("patient_id", "onset_date", "organization_id")])

We can see that there are a few duplicate entries just in the head of this joint data frame. This is because one patient could have multiple pre-existing conditions, each of which could be associated with care at a different organization. In other words, duplicates are perfectly okay for now, but we'll have to keep this in mind when we get to some of our analytics examples below.

### Answering Exploratory Profiling Questions

Now that we've created joint data sets, we want to start exploring the data to answer questions. Some of the most basic exploration questions we're likely to encounter involve profiling data along demographic fields. For example: what are the relative ratios of each gender in this data set? We can easily answer this question by performing some counting analysis on our data.

In [ ]:
# Create an aggregate count of a one-variable field in the data
# Use mutate to turn this into a percentage for clear visualization
counts <- patients %>% 
    group_by(gender) %>%
    count() %>% 
    ungroup() %>% 
    mutate(perc = `n` / sum(`n`)) %>% 
    arrange(perc) %>%
    mutate(labels = scales::percent(perc))

# A pie chart is a helpful way of visualizing this information
ggplot(counts, aes(x = "", y = perc, fill = gender)) +
    geom_col() +
    geom_text(aes(label = labels), position = position_stack(vjust = 0.5)) +
    coord_polar(theta = "y")

### Time Series Analysis

Now that we've answered a simple question, let's look a little deeper. Suppose we're interested in the onset of various medical problems over time. We want to see if there are any identifiable trends in the number of conditions that patients in the data have reported. Our pulled down data allows us to easily answer this question as well. We'll start by creating a more generalized variable for the year of a condition's onset, instead of the precise date, in order to facilitate counting statistics. Then, it's just a matter of grouping and plotting.

In [ ]:
# Extract the year of a condition's onset into its own variable to enable counting
# Also eliminate rows where a condition is missing time information
conditions <- conditions[!(is.na(conditions$onset_date) | conditions$onset_date == ""),]
conditions$onset_year <- conditions$onset_date %>%
    map(str_split, pattern = "-") %>%
    map(pluck(1)) %>%
    map(pluck(1)) %>%
    unlist()    # Need to invoke this because map always returns a list

# Tabulate stats on number of occurences
counts <- conditions %>%
    group_by(onset_year) %>%
    count() %>%
    ungroup() %>%
    arrange(onset_year)

# And plot
ggplot(counts, aes(x = onset_year, y = n, group = 1)) +
    geom_point() +
    geom_line() +
    labs(x = "Reporting Year", y = "# of Patients With Condition Onset in Year", title = "Condition Onset Over Time")

### A Fake Status Histogram

Next, let's make a toy assumption for the purpose of demonstration. Let's create a "fake" status for each patient in our data by treating the last digit of their phone number as if it was indicative of some sort of status or test (this is obviously completely random, but it helps bear out the graph). We'll begin by extracting this last digit into its own "status" field, and then we'll visualize a histogram of the results.

In [ ]:
# Extract last digit of phone number into its own field and convert to numeric
patients_with_phones <- patients[!(is.na(patients$phone_number) | patients$phone_number == ""),]
patients_w_status <- patients_with_phones %>%
  mutate(
    status=as.numeric(stringr::str_sub(phone_number,-1,-1))
  )

# Now let's visualize what that looks like
ggplot(patients_w_status) +
    geom_histogram(aes(x=status),fill="green",color="white",bins=10) +
    ggtitle("Patient Status Test") + theme_minimal()

### A Choropleth Visualization Of Patients

For our final example, let's do something a little more applicable. Using the joined data we created in the first example above, let's create a choropleth visualization of this data disaggregated across the state of Virginia (a _choropleth_ chart is a map where each region is color-coded by intensity of some numerical variable, like a heatmap). We'll be using the most 2018 tracts information on counties and subcounties published by Virginia itself, available [here](https://catalog.data.gov/dataset/tiger-line-shapefile-2018-state-virginia-current-census-tract-state-based). While it's possible to use data formats such as GeoJSON for spatial analysis, we'll opt for a shapefile, as that's the preferred layer format used by most US Federal agencies and surveys (such as the Census). We'll be performing only a basic analysis, creating a choropleth for all patients in our dataset with preexisting conditions, but the points made below are applicable to a wide variety of geospatial applications.

Let's start by reading in our geospatial data for the state of Virginia and visualizing what the map looks like when blank. To load the data, we'll just have to replace the bracketed text below with the absolute path to our tracts shapefile. The blank map visualization will take a moment or two to generate, so we'll just have to be patient while it runs.

In [ ]:
# Open the shapefile using the sf package
va_tracts = st_read(PATH_TO_TRACT_FILES)
# Geodetic CRS is the Coordinate Reference System that encodes the multipolygons
# NAD83 Corresponds to code US 4269, which is the CRS most commonly used by 
# Federal Agencies (https://www.nceas.ucsb.edu/sites/default/files/2020-04/OverviewCoordinateReferenceSystems.pdf)

In [ ]:
head(va_tracts)

In [ ]:
ggplot() + 
  geom_sf(data = va_tracts) + 
  ggtitle("Spatial Decomposition of Virginia") + 
  coord_sf()

We can see a good projection of the various counties and tracts in the state. There are even some pockets so dense with drawing that it looks colored-in (but this is just an artifact of the drawing process in such close proximity). 

Let's identify the patients in our joint dataset that have preexisting conditions. We can do this using some simple filtering techniques, assuming that if the `onset_date` column from the `Conditions` table is blank, then there is no condition associated with that patient. Also, recall that we observed duplicates in our data frame after joining, representing the possibility of patients with multiple conditions. To account for this, we'll only consider distinct patients and their conditions by geography.

In [ ]:
patients_with_conditions <- joined_df[!(joined_df$onset_date == "" | is.na(joined_df$onset_date)),]
de_duped <- patients_with_conditions %>% distinct(patient_id, onset_date, latitude, longitude)

# Also drop patients lacking geographical information
de_duped <- de_duped[!(is.na(de_duped$longitude) | is.na(de_duped$latitude)),]

nrow(joined_df)
nrow(de_duped)

Taking distinct values along these dimensions cut the size of our joint data set by an order of magnitude! Now that we have our distinct patients and their conditions, we can use the `sf` package to place them geographically within VA's tract plots using `point-in-polygon` functionality. This will allow us to join our two dataframes on a common identifier, which will let us visualize the map filled-in with color.

In [ ]:
# Convert base patient data into a spatial-features data for point placement
geolocated_patients <- de_duped %>%
    mutate_at(vars(longitude, latitude), as.numeric) %>%   # coordinates must be numeric
    st_as_sf(
        coords = c("longitude", "latitude"),
        agr = "constant",
        crs = 4269,        # nad83
        stringsAsFactors = FALSE,
        remove = TRUE
    )

# Use spatial join to place patients into the correct tracts
patient_in_tract <- st_join(geolocated_patients, va_tracts, join=st_within)
tract_count <- count(as_tibble(patient_in_tract), NAME)

# Before passing the data into visualize, in order for ggplot to map the numeric values
# with geographic localities, the spatial features data must be "transformed" into an
# absolute data frame structure, so we'll call `fortify()` here to do just that
choro_df <- left_join(fortify(va_tracts), tract_count, by = "NAME") %>%
    rename(patients_with_conditions = n)

# And let's see what it looks like!
ggplot(choro_df, aes(fill = patients_with_conditions)) + 
    geom_sf() + 
    scale_fill_distiller(
        type = "seq",
        palette = "Blues",
        na.value = "transparent",
        direction = 1
    ) +
    theme_void() +
    ggtitle("Spatial Decomposition of Virginia")